### 종합실습2
##### Requests 혹은 Selenium 모듈을 이용해 다이닝코드 페이지로 접속한 다음, 포항 지역의 맛집 100건을 검색하여 엑셀로 저장해 보세요. (다이닝코드 검색어는 '포항 맛집'으로 하세요)
##### 링크 : https://www.diningcode.com/

### 1. Requests로 크롤링하기

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#SSL Verify 경고 끄기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

#아래에 코드 작성
results = []
for page_no in range(1, 11):
    print("{}페이지 크롤링 중...".format(page_no), end="\r")
    url = "https://www.diningcode.com/2018/ajax/list.php?query=%ED%8F%AC%ED%95%AD%20%EB%A7%9B%EC%A7%91&page="+str(page_no)
    response = requests.get(url, verify = False)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    items = soup.select(".blink")
    for item in items:
        this_result_dict = {
            "이름" : item.select(".btxt")[0].text.split(". ")[1],
            "종류" : item.select(".stxt")[0].text,
            "특징" : item.select(".ctxt")[0].text
        }
        if len(item.select(".ctxt")[1].select("i"))>0:
            this_result_dict["위치"] = item.select(".ctxt")[1].select("i")[0].text
            item.select(".ctxt")[1].select("i")[0].decompose()
        this_result_dict["주소"] = item.select(".ctxt")[1].text
        results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항 맛집 리스트.xlsx", index = False)
display(df)

### 2. Selenium으로 크롤링하기

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

#아래에 코드 작성
driver = webdriver.Chrome("/Users/hrdkdh/Downloads/chromedriver")
driver.get("https://www.diningcode.com/")
driver.find_element_by_css_selector("#txt_keyword").send_keys("포항 맛집")
driver.find_element_by_css_selector("#txt_keyword").submit()
driver.find_element_by_css_selector("#btn_normal_list").click()

for i in range(0, 10): #더보기 버튼을 눌러 추가로 데이터를 로드함. 10번 반복하여 총 100개의 데이터 로드
    more_btn = driver.find_element_by_css_selector("#div_list_more > .more-btn")
    try:
        more_btn.click()
    except:
        print("모든 데이터를 로드하였습니다.")
    time.sleep(2) #데이터 로드시간 편차가 있어 2초 정도 쉬어 줌

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
items = soup.select(".blink")
results = []
for item in items:
    this_result_dict = {
        "이름" : item.select(".btxt")[0].text.split(". ")[1],
        "종류" : item.select(".stxt")[0].text,
        "특징" : item.select(".ctxt")[0].text
    }
    if len(item.select(".ctxt")[1].select("i"))>0:
        this_result_dict["위치"] = item.select(".ctxt")[1].select("i")[0].text
        item.select(".ctxt")[1].select("i")[0].decompose()
    this_result_dict["주소"] = item.select(".ctxt")[1].text
    results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항 맛집 리스트.xlsx", index = False)
display(df)